In [3]:
# Imports
import numpy as np
import ufl
import sys, slepc4py
slepc4py.init(sys.argv)

from dolfinx import fem
from dolfinx.fem import (Constant, dirichletbc, Function,
        locate_dofs_topological)
from dolfinx.mesh import (create_box, CellType,
        locate_entities_boundary)
from dolfinx.io import XDMFFile
from mpi4py import MPI
from slepc4py import SLEPc
from dolfinx.fem.petsc import assemble_vector, assemble_matrix
from ufl import VectorElement

# Define computational domain
L = np.array([5, 0.6, 0.4])
N = [25, 3, 2]
mesh = create_box(MPI.COMM_WORLD, [np.array([0,0,0]), L], N,
        cell_type=CellType.hexahedron)
        
# Material constants
E, nu = (2e11), (0.3)  
rho = (7850) 
mu = Constant(mesh, E/2./(1+nu))
lambda_ = Constant(mesh, E*nu/(1+nu)/(1-2*nu))

# Convenience functions
def epsilon(u):
    return ufl.sym(ufl.grad(u))
def sigma(u):
    return lambda_ * ufl.nabla_div(u) * ufl.Identity(3) + 2*mu*epsilon(u)

# Define function space, trial and test functions
Ve = VectorElement("Lagrange", mesh.ufl_cell(), 1)
V = fem.FunctionSpace(mesh, Ve)
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Define Dirichlet boundary condition
def clamped_boundary(x):
    return np.isclose(x[0], 0)

fdim = mesh.topology.dim - 1
boundary_facets = locate_entities_boundary(mesh, fdim, clamped_boundary)

u_D = Function(V)
u_D.interpolate(lambda x: x * 0)
bc = dirichletbc(u_D, locate_dofs_topological(V, fdim, boundary_facets))

# Define variational form
k_form = ufl.inner(sigma(u),epsilon(v))*ufl.dx
m_form = rho*ufl.inner(u,v)*ufl.dx

# Assemble stiffness and mass matrices
#
# Using the "diagonal" kwarg ensures that Dirichlet BC modes will not be among
# the lowest-frequency modes of the beam. 
M = assemble_matrix(fem.form(m_form), bcs=[bc], diagonal=1/62831)
K = assemble_matrix(fem.form(k_form), bcs=[bc], diagonal=62831)

K.assemble()
M.assemble()

# Create and configure eigenvalue solver
N_eig = 6
eigensolver = SLEPc.EPS().create(MPI.COMM_WORLD)
eigensolver.setDimensions(N_eig)
eigensolver.setProblemType(SLEPc.EPS.ProblemType.GHEP)
st = SLEPc.ST().create(MPI.COMM_WORLD)
st.setType(SLEPc.ST.Type.SINVERT)
st.setShift(0.1)
st.setFromOptions()
eigensolver.setST(st)
eigensolver.setOperators(K, M)
eigensolver.setFromOptions()

# Compute eigenvalue-eigenvector pairs
eigensolver.solve()
evs = eigensolver.getConverged()
vr, vi = K.getVecs()
u_output = Function(V)
u_output.name = "Eigenvector"
print( "Number of converged eigenpairs %d" % evs )
if evs > 0:
    with XDMFFile(MPI.COMM_WORLD, "eigenvectors.xdmf", "w") as xdmf:
        xdmf.write_mesh(mesh)
        for i in range (min(N_eig, evs)):
            l = eigensolver.getEigenpair(i, vr, vi)
            freq = np.sqrt(l.real)/2/np.pi
            print(f"Mode {i}: {freq} Hz")
            u_output.x.array[:] = vr
            xdmf.write_function(u_output, i)

ld: warning: duplicate -rpath '/opt/homebrew/Caskroom/miniforge/base/envs/femenv/lib' ignored
ld: warning: duplicate -rpath '/opt/homebrew/Caskroom/miniforge/base/envs/femenv/lib' ignored
ld: warning: duplicate -rpath '/opt/homebrew/Caskroom/miniforge/base/envs/femenv/lib' ignored


Number of converged eigenpairs 8
Mode 0: 13.946300071891583 Hz
Mode 1: 20.084392521145215 Hz
Mode 2: 85.25303934738459 Hz
Mode 3: 118.83155190127196 Hz
Mode 4: 141.79868660656626 Hz
Mode 5: 230.52964645985134 Hz


In [1]:

from mpi4py import MPI
import numpy as np
from petsc4py import PETSc
from slepc4py import SLEPc
from dolfinx.io import XDMFFile
from dolfinx import mesh, fem
from dolfinx.fem import Constant,Function
import ufl
from ufl import TrialFunction, TestFunction, sym, grad, tr, Identity, inner, dx
from ufl import VectorElement

# -------------------------
# Geometry (3D block)
# -------------------------
Lx, Ly, Lz = 1.0, 0.1, 0.1
nx, ny, nz = 20, 10,10

domain = mesh.create_box(
    MPI.COMM_WORLD,
    np.array([[0.0, 0.0, 0.0],
             [Lx,  Ly,  Lz]]),
    [nx, ny, nz],
    cell_type=mesh.CellType.tetrahedron
)
# -------------------------
# Function space (3D displacement)
# -------------------------
Ve = VectorElement("Lagrange", domain.ufl_cell(), 1)
V = fem.FunctionSpace(domain, Ve)

u = TrialFunction(V)
v = TestFunction(V)

xm0, xm1 = 0.5, 0.55
ym0, ym1 = 0.01, 0.05
zm0, zm1 = 0, 0.01
# -------------------------
# Material (3D linear elasticity)
# -------------------------
E = 210e9
nu = 0.3
rho = 7850.0

mu = Constant(domain, E/2./(1+nu))
lambda_ = Constant(domain, E*nu/(1+nu)/(1-2*nu))

# Convenience functions
def eps(u):
    return ufl.sym(ufl.grad(u))
def sigma(u):
    return lambda_ * ufl.nabla_div(u) * ufl.Identity(3) + 2*mu*eps(u)

# Stiffness
a = inner(sigma(u), eps(v)) * dx
a_form = fem.form(a)

# Mass
m = rho * inner(u, v) * dx
m_form = fem.form(m)
# -------------------------
# Clamp boundary: x = 0 face
# -------------------------
def clamp_region(x):
    return (
        np.isclose(x[0], 0.5, atol=0.01) &  # 중앙 x=0.5 ± 1cm
        np.isclose(x[2], 0, atol=0.01)  # 바닥
    )

dofs_clamp = fem.locate_dofs_geometrical(V, clamp_region)

zero = np.array([0.0, 0.0, 0.0], dtype=PETSc.ScalarType)
bc = fem.dirichletbc(zero, dofs_clamp, V)
bcs = [bc]
# -------------------------
# Assemble matrices
# -------------------------
K = fem.petsc.assemble_matrix(a_form, bcs=bcs, diagonal=1/62831)
K.assemble()

M = fem.petsc.assemble_matrix(m_form, diagonal=62831)
M.assemble()

# -------------------------
# Lumped mass block (rectangular volume)
# -------------------------
xm0, xm1 = 0.05, 0.1
ym0, ym1 = 0.01, 0.05
zm0, zm1 = 0, 0.01

m_add = 10.0  # kg (총 추가 질량)
def in_mass_block(x):
    return ((x[0] >= xm0) & (x[0] <= xm1) &
            (x[1] >= ym0) & (x[1] <= ym1) &
            (x[2] >= zm0) & (x[2] <= zm1))
dofs_mass = fem.locate_dofs_geometrical(V, in_mass_block)

if len(dofs_mass) == 0:
    raise RuntimeError("Mass block DOFs not found")
ndofs = len(dofs_mass)
m_per_dof = m_add / ndofs

for dof in dofs_mass:
    M.setValue(dof, dof, m_per_dof, addv=True)

M.assemble()
# Create and configure eigenvalue solver
N_eig = 90
eigensolver = SLEPc.EPS().create(MPI.COMM_WORLD)
eigensolver.setDimensions(N_eig)
eigensolver.setProblemType(SLEPc.EPS.ProblemType.GHEP)
st = SLEPc.ST().create(MPI.COMM_WORLD)
st.setType(SLEPc.ST.Type.SINVERT)
st.setShift(50)
st.setFromOptions()
eigensolver.setST(st)
eigensolver.setOperators(K, M)
eigensolver.setFromOptions()

# Compute eigenvalue-eigenvector pairs
eigensolver.solve()
evs = eigensolver.getConverged()
vr, vi = K.getVecs()
u_output = Function(V)
u_output.name = "Eigenvector"
print( "Number of converged eigenpairs %d" % evs )

Number of converged eigenpairs 116


In [4]:
import numpy as np
from petsc4py import PETSc
from dolfinx import fem
import ufl

freq = 120.0
omega = 2*np.pi*freq
a_base = 1.0       # m/s^2
zeta = 0.01

ndofs = M.getSize()[0]
bs = V.dofmap.index_map_bs

# ---- 베이스 영향 벡터 b (z방향) ----
b = PETSc.Vec().createMPI(ndofs, comm=M.comm)
b.set(0.0)

clamp_set = set(map(int, dofs_clamp))
r0, r1 = b.getOwnershipRange()
for dof in range(r0, r1):
    if (dof % bs) == 2 and (dof not in clamp_set):
        b.setValue(dof, 1.0)

b.assemblyBegin(); b.assemblyEnd()

Mb = M.createVecRight()
M.mult(b, Mb)

# ---- 모달 중첩 ----
u_resp = np.zeros(ndofs, dtype=np.complex128)
modes = []     # ← 이게 modes
freqs = []     # 고유진동수도 같이 저장

for i in range(evs):
    l = eigensolver.getEigenpair(i, vr, vi)

    # 고유치 → 주파수
    if l.real <= 0:
        continue

    freq = np.sqrt(l.real) / (2*np.pi)

    # 저주파(강체모드) 컷
    if freq < 5.0:
        continue

    # PETSc Vec → dolfinx Function
    mode = Function(V)
    mode.x.array[:] = vr.array[:]   # ⭐ 핵심
    mode.name = f"mode_{i}"

    modes.append(mode)
    freqs.append(freq)

    print(f"Mode {i}: {freq:.2f} Hz")


omega_r = 2*np.pi*np.array(freqs)   # rad/s
for r, mode in enumerate(modes):
    phi = PETSc.Vec().createWithArray(mode.x.array, comm=M.comm)
    Fr = - phi.dot(Mb) * a_base
    den = (omega_r[r]**2 - omega**2) + 2j*zeta*omega_r[r]*omega
    u_resp += (mode.x.array * Fr) / den

# clamp DOF는 상대변위 0
u_resp[np.array(dofs_clamp, dtype=np.int32)] = 0.0


Mode 66: 122.28 Hz
Mode 67: 266.84 Hz
Mode 68: 363.13 Hz
Mode 69: 374.35 Hz
Mode 70: 1412.59 Hz
Mode 71: 1503.38 Hz
Mode 72: 1700.69 Hz
Mode 73: 1858.58 Hz
Mode 74: 1901.69 Hz
Mode 75: 1976.42 Hz
Mode 76: 2229.05 Hz
Mode 77: 2557.77 Hz
Mode 78: 3765.10 Hz
Mode 79: 4328.21 Hz
Mode 80: 4418.23 Hz
Mode 81: 4598.71 Hz
Mode 82: 5110.38 Hz
Mode 83: 5673.70 Hz
Mode 84: 6445.81 Hz
Mode 85: 6849.50 Hz
Mode 86: 7192.35 Hz
Mode 87: 7503.49 Hz
Mode 88: 7844.60 Hz
Mode 89: 8052.90 Hz
Mode 90: 8600.01 Hz
Mode 91: 9153.66 Hz
Mode 92: 9464.76 Hz
Mode 93: 10028.80 Hz
Mode 94: 10280.99 Hz
Mode 95: 11187.45 Hz
Mode 96: 11427.54 Hz
Mode 97: 11538.66 Hz
Mode 98: 11887.27 Hz
Mode 99: 12900.35 Hz
Mode 100: 13209.35 Hz
Mode 101: 13500.63 Hz
Mode 102: 13690.37 Hz
Mode 103: 14261.37 Hz
Mode 104: 15064.30 Hz
Mode 105: 15211.31 Hz
Mode 106: 16105.12 Hz
Mode 107: 16386.69 Hz
Mode 108: 16544.92 Hz
Mode 109: 17058.01 Hz
Mode 110: 17246.54 Hz
Mode 111: 17294.92 Hz
Mode 112: 17628.30 Hz
Mode 113: 17944.78 Hz
Mode 114:

In [5]:
u_real = fem.Function(V)
u_real.name = "u_real_120Hz"
u_real.x.array[:] = np.real(u_resp)
We = ufl.TensorElement("DG", domain.ufl_cell(), degree=0, shape=(3, 3))
W = fem.FunctionSpace(domain, We)
# W = fem.FunctionSpace(domain, ("DG", 0, (3,3)))   # cell-wise stress
stress_cell = fem.Function(W)
stress_expr = fem.Expression(sigma(u_real), W.element.interpolation_points())
stress_cell.interpolate(stress_expr)
